In [2]:
import re
book='matt'
booklong='matthew'
inF=open("txts/"+book+".txt","r")
test=inF.read()

In [3]:
t=re.sub('(\([^\)]*)(\d+)([\D]*\))','',test)
chs=t.split('|')
len(chs)

28

In [4]:
#len(chs[0].trim())
import json, time ,re

In [13]:
#create index for chapters on elasticsearch

import elasticsearch
index_name = "bchaps"
index_type = "bible_chapters"
elastic_host = "gaia.hud.ac.uk"
elastic_port = "9200"
# Connect to ElasticSearch
es = elasticsearch.Elasticsearch(elastic_host, port=elastic_port)
# Delete & Create index
es.indices.delete(index=index_name, ignore=[400, 404])
es.indices.create(index=index_name, ignore=400)

{'acknowledged': True, 'index': 'bchaps', 'shards_acknowledged': True}

In [14]:
chn=[]
for i in range(len(chs)):
    if i>0:
        chno=int(re.match('^\d+',chs[i]).group())
        cno=str(chno).zfill(2)
        verx=chs[i][re.match('^\d+',chs[i]).end():]
        verses=re.findall('\D+',chs[i])
        vnos=re.findall('\d+',chs[i])
        vnos[0]="1"
        mismatch=len(vnos)!=len(verses)
        import numpy as np
        x = np.array(vnos)
        vnos = x.astype(np.int).tolist()
        js={
            "bookcode":book, 
            "book":booklong,
            "chno":chno,
            "vnos":vnos,
            "verses":verses,
            "mismatched":mismatch,
            "text":verx
        }
        data=json.dumps(js)
        #print(data)
        #POST TO ElasticSearch
        res = es.index(index=index_name, doc_type=index_type, id=book+'_'+cno, body=data)
        #print(res)
        print(book+'_'+cno, "..created!" if res['_shards']['successful']>0 else "failed")
        #time.sleep(1.0)

matt_01 ..created!
matt_02 ..created!
matt_03 ..created!
matt_04 ..created!
matt_05 ..created!
matt_06 ..created!
matt_07 ..created!
matt_08 ..created!
matt_09 ..created!
matt_10 ..created!
matt_11 ..created!
matt_12 ..created!
matt_13 ..created!
matt_14 ..created!
matt_16 ..created!
matt_17 ..created!
matt_18 ..created!
matt_19 ..created!
matt_20 ..created!
matt_21 ..created!
matt_22 ..created!
matt_23 ..created!
matt_24 ..created!
matt_25 ..created!
matt_26 ..created!
matt_27 ..created!
matt_28 ..created!


In [16]:
es.get(index='bchaps', doc_type='bible_chapters', id='matt_20')['_source']

{'book': 'matthew',
 'bookcode': 'matt',
 'chno': 20,
 'mismatched': False,
 'text': ' “Sopbie Amanyanakiri mi Vain ozu nyanabo dede bie pakamun oria vain ozu bie firnwengibia apu oko-e munoku simeme. 2 Ori ini na bipi okiame na gbori kobo mi oe enetibi bu bobo gbeinbia ye-e, se ini so oria vain ozu mi bie mume, firinwengibia ereche. 3 Esenie-a so biri bu o piki paka feschiri mun se firi nyana ka jin owuapu nwọ orime. 4 Se O nwose ini beme, ‘Omine soni o mun vain ozu mi bie firinwengi, se o gbein na ibi bara a nwọse o gbeinbia.’ 5 Ani ini mun ye-e. Onyma so finiji biri bun a, piki tere-a so biri bun a, ori ani gbori bara mi nwọ mieme 6 Ani sonio-a so biri nwọ la-e ton-abe, se o piki fechiri mi nwọ mune, se piki ton-a owupau nwọ orime, ani nyo chie sime warabe. O nwọse ini gbelame, ‘Nde o gose minanga bu bebe ene mimgba nwọ famari ye, mieari gboriye so ofori bara bu? 7 Ini nwọ pekereme. ‘Gboribo soni firi bu mine okoke.’ O nwọse ini beme, ‘Ibime, ani-e bebe, omine soni o mun vain ozu mi

In [17]:
es.search(index="bchaps", body={"query": {"match": {'mismatched':True}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [], 'max_score': None, 'total': 0},
 'timed_out': False,
 'took': 5}

In [18]:
es.search(index="bchaps", body={"query": {"match": {'mismatched':False}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'matt_11',
    '_index': 'bchaps',
    '_score': 0.105360515,
    '_source': {'book': 'matthew',
     'bookcode': 'matt',
     'chno': 11,
     'mismatched': False,
     'text': ' Jizos Be maa datekerema ma nwose oria obuduko oyimapu finji ma piriso siki, ori ani nyo mi koromaso, se nwengi mun ani na kpeki ama ma bie yedie, piki Tamuno bere mi dukome.\n2 Jon, mengisarabo be, ikoliwaris bie sime Kraist be nwengiari firi ma nwo naa siki, ori oria obudukoapu ma be dapu se o firimame. 3Se nwose Jizos Be gbelame, “Duko wa piri,” Jon be bere be bobia nwo be bo, “Iri mono, se wa toru se jinbo diki-au?”\n4 Jizos Be ini pekereme, “O vinpiki mun o naasa ye ma na, omine orisa ye ma na duko Jon be piri: 5Torumgbolupun-apu ma inia torumgbolu ma finjisam, ikeliapu ma ini nwengisam. Bukan-obi kon-apu ma ini nwo mie fiafiamasam, inikiapu ma ini ye naasam, fiapu ma ini sisesam, ini piki Ibi Bere mi duko igw